In [2]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
#import yfinance as yf
import re
import calendar
from fii import FII, csv_para_df, fii_preco

ImportError: cannot import name 'csv_para_df' from 'fii' (d:\OneDrive\Trabalho\LIS\Péricles\Python\FII\fii.py)

In [20]:
import pandas as pd
import plotly.graph_objs as go

# Criando um exemplo de dataframe
dados = {
    'data': ['2023-01-15', '2023-02-20', '2023-03-10', '2024-01-05', '2024-02-12'],
    'proventos': [1000.50, 1500.75, 800.30, 1200.60, 1700.80]
}

rendimentos = pd.DataFrame(dados)
rendimentos['data'] = pd.to_datetime(rendimentos['data'])

# Filtrando os dados por ano
rendimentos_2023 = rendimentos[rendimentos['data'].dt.year == 2023]
rendimentos_2024 = rendimentos[rendimentos['data'].dt.year == 2024]

# Calculando a soma dos proventos por mês para cada ano
rendimentos_2023_agrupados = rendimentos_2023.groupby(rendimentos_2023['data'].dt.month)['proventos'].sum()
rendimentos_2024_agrupados = rendimentos_2024.groupby(rendimentos_2024['data'].dt.month)['proventos'].sum()

# Mapeando números de meses para nomes de meses
meses = {
    1: 'Jan',
    2: 'Fev',
    3: 'Mar',
    4: 'Abr',
    5: 'Mai',
    6: 'Jun',
    7: 'Jul',
    8: 'Ago',
    9: 'Set',
    10: 'Out',
    11: 'Nov',
    12: 'Dez'
}

# Criando o gráfico de barras
trace1 = go.Bar(
    x=[meses[mes] for mes in rendimentos_2023_agrupados.index],
    y=rendimentos_2023_agrupados.values,
    name='2023',
    marker=dict(color='orange'),  # Definindo a cor laranja para as barras de 2023
    hoverinfo='text',
    text=[f'Proventos: R${valor:.2f}<br>Ano: 2023' for valor in rendimentos_2023_agrupados.values],
    textposition='outside'  # Colocando o texto fora das barras
)
trace2 = go.Bar(
    x=[meses[mes] for mes in rendimentos_2024_agrupados.index],
    y=rendimentos_2024_agrupados.values,
    name='2024',
    hoverinfo='text',
    text=[f'Proventos: R${valor:.2f}<br>Ano: 2024' for valor in rendimentos_2024_agrupados.values],
    textposition='outside'  # Colocando o texto fora das barras
)

layout = go.Layout(
    title='Proventos por mês',
    xaxis=dict(title='Mês'),
    yaxis=dict(title='Proventos')
)

fig = go.Figure(data=[trace1, trace2], layout=layout)

# Plotando o gráfico
fig.show()


In [11]:
fii_codigo = 'GGRC11'

ano = 2024
mes = 1

fii = FII(fii_codigo, ano, mes)

try:
    informes = fii.fii_informe()
    informes.to_csv(f'./csv/{fii_codigo}_informes_{ano}_{mes}.csv', sep=';', index=False)
except:
    pass

In [6]:
import locale

def formato(valor):
    locale.setlocale(locale.LC_ALL, 'pt_BR.UTF-8')
    return locale.currency(valor, grouping=True)

formato(3445982046.45)

'R$ 3.445.982.046,45'

In [3]:
fiis = ['XPML11']

anos = [2023]
meses = [12, 11, 10, 9, 8, 7, 6, 5, 4, 3, 2, 1]

for fii_codigo in fiis:
    for ano in anos:
        for mes in meses:
            fii = FII(fii_codigo, ano, mes)



            try:
                informes = fii.fii_informe()
                informes.to_csv(f'./csv/{fii_codigo}_informes_{ano}_{mes}.csv', sep=';', index=False)
            except:
                pass

In [3]:
import yfinance as yf
import plotly.graph_objects as go

def grafico_preco(fii_codigo):
    ticker = fii_codigo+'.SA'

    hist = yf.download(ticker, period="52wk")
    atual = hist['Close'].iloc[-1]

    fig = go.Figure()

    fig.add_trace(go.Scatter(x=hist.index, y=hist['Close'], mode='lines', name='Preço de Fechamento'))

    fig.add_trace(go.Scatter(x=[hist.index[-1]], y=[atual], mode='markers', marker=dict(color='red'), name='Preço Atual'))

    fig.update_layout(title='Preço atual e dia correspondente',
                    xaxis_title='Data',
                    yaxis_title='Preço (R$)')

    return fig.show()


In [12]:
grafico_preco('XPML11')

[*********************100%%**********************]  1 of 1 completed


In [14]:
class FII:

    def __init__(self, fii_codigo, ano, mes):
        self.fii_codigo = fii_codigo
        self.ano = ano
        self.mes = mes

    def ultimo_dia(self, ano, mes):
        return calendar.monthrange(ano, mes)[1]

    def noticias(self):
        ultimo_dia = self.ultimo_dia(self.ano, self.mes)
        return f"https://sistemasweb.b3.com.br/PlantaoNoticias/Noticias/ListarTitulosNoticias?agencia=18&palavra={self.fii_codigo[:4]}&dataInicial={self.ano}-{self.mes}-01&dataFinal={self.ano}-{self.mes}-{ultimo_dia}"

    def json(self):
        response = requests.get(self.noticias())
        return response.json()
    
    def json_item(self, texto):
        json_item = []
        json_relatorios = self.json()
        for item in json_relatorios:
            headline = item['NwsMsg']['headline']
            if texto in headline:
                json_item .append(item)
        return json_item[0]
    
    def url_noticia(self, texto):
        json_item = self.json_item(texto)
        item_IdNoticia = json_item['NwsMsg']['id']
        item_IdAgencia = json_item['NwsMsg']['IdAgencia']
        item_dateTime = json_item['NwsMsg']['dateTime'].replace(' ', '%20')
        url = f'https://sistemasweb.b3.com.br/PlantaoNoticias/Noticias/Detail?idNoticia={item_IdNoticia}&agencia={item_IdAgencia}&dataNoticia={item_dateTime}'
        pattern = r'dataNoticia=(.*?)%20'
        matches = re.findall(pattern, url)
        return url, matches

    def url_documento(self, texto):
        url = self.url_noticia(texto)
        response = requests.get(url)
        html_content = response.text
        soup = BeautifulSoup(html_content, 'html.parser')
        pre_element = soup.find('pre', id='conteudoDetalhe')
        pre_text = pre_element.get_text()
        padrao = r'(https?://[^&]+)'
        resultado = re.search(padrao, pre_text)
        url = resultado.group(1).replace('visualizarDocumento', 'exibirDocumento').split('&')[0]
        return url
    
    def soup(self, texto):
        url = self.url_documento(texto)
        response = requests.get(url)
        soup = BeautifulSoup(response.text, "html.parser")
        return soup
    
    def soup_webdriver(self, texto):
        url = self.url_documento(texto)
        driver = webdriver.Chrome()
        driver.get(url)
        driver.implicitly_wait(5)
        html = driver.page_source
        driver.quit()
        return BeautifulSoup(html, 'html.parser')
    
    def html(self, texto, elementos):
        html = self.soup_webdriver(texto).find_all(elementos)
        return html
    
    def tabelas(self, texto, elementos):
        tabelas = self.html(texto, elementos)
        return tabelas
    
    def indice(self, tabela, expressao):
        indice = None
        for i, linha in enumerate(tabela):
            if expressao in linha:
                indice = i
                break
        return indice
    
    def str_para_float(self, valor):
        return float(valor.replace(".", "").replace(",", "."))
    
    def str_para_int(self, valor):
        return int(float(valor.replace(".", "").replace(",", ".")))
    
    def tabela(self, i, tabelas_fii):
            tabela = tabelas_fii[i].find_all('td')
            info = [item.text for item in tabela]
            info = [info.strip() for info in info]
            return info
    
    def fii_informe(self):
        texto = 'Informe Mensal'
        elementos = 'table'
        tabelas_fii = self.tabelas(texto, elementos)
        
        df_dados_fundos = self.tabela(0, tabelas_fii)
        df_cotistas = self.tabela(1, tabelas_fii)
        df_ativos = self.tabela(2, tabelas_fii)
        df_ativos2 = self.tabela(3, tabelas_fii)
        df_passivos = self.tabela(4, tabelas_fii)

        pattern = r'Mandato:\s*</b>(.*?)</span><span class="dado-cabecalho"><b>Segmento de Atuação:\s*</b>(.*?)</span><span class="dado-cabecalho"><b>Tipo de Gestão:\s*</b>(.*?)</span></td>'
        match = re.search(pattern, str(tabelas_fii[0]))

        df_fii = pd.DataFrame({
            'codigo_isin': [df_dados_fundos[self.indice(df_dados_fundos, 'Código ISIN')+1]],
            'cotistas': [self.str_para_int(df_cotistas[self.indice(df_cotistas, 'Número de cotista')+1])],
            'patrimonio_liquido': [self.str_para_float(df_ativos[self.indice(df_ativos, 'Patrimônio Líquido')+1])],
            'cotas_emitidas': [self.str_para_int(df_ativos[self.indice(df_ativos, 'Número de Cotas Emitidas')+1])],
            'valor_patrimonial_cota': [self.str_para_float(df_ativos[self.indice(df_ativos, 'Valor Patrimonial das Cotas')+1])],
            'valor_em_caixa': [self.str_para_float(df_ativos2[self.indice(df_ativos2, 'Total mantido para as Necessidades de Liquidez')+1])],
            'patrimonio_liquido': [self.str_para_float(df_ativos[self.indice(df_ativos, 'Patrimônio Líquido')+1])],
            'total_investido': [self.str_para_float(df_ativos2[self.indice(df_ativos2, 'Total investido')+1])],
            'passivo': [self.str_para_float(df_passivos[self.indice(df_passivos, 'Total do passivo')+1])],
            'mandato': [match.group(1).strip()],
            'seguimento': [match.group(2).strip()],
            'tipo_gestão': [match.group(3).strip()],
            'competencia': [df_dados_fundos[self.indice(df_dados_fundos, 'Competência')+1]],
        })        
        return df_fii
    
    def fii_rendimento(self):
        texto = 'Aviso aos Cotistas'
        elementos = 'table'
        tabelas_fii = self.tabelas(texto, elementos)

        df_dados_fundos = self.tabela(0, tabelas_fii)
        proventos = self.tabela(1, tabelas_fii)

        df_fii = pd.DataFrame({
            'codigo_isin': [proventos[self.indice(proventos, 'Código ISIN')+1]],
            'codigo_negociacao': [proventos[self.indice(proventos, 'Código de negociação')+1]],
            'proventos': [self.str_para_float(proventos[self.indice(proventos, 'Valor do provento')+1])],
            'data_base': [proventos[self.indice(proventos, 'Data-base')+1]],
            'data_pagamento': [proventos[self.indice(proventos, 'Data do pagamento')+1]],
            'periodo': [proventos[self.indice(proventos, 'Período de referência')+1]],
        })

        return df_fii
    
    def fii_relatorios(self):
        relatorios = self.json()
        df = pd.json_normalize(relatorios)
        df['dia_hora'] = df['NwsMsg.dateTime']
        df['NwsMsg.dateTime'] = df['NwsMsg.dateTime'].str.replace(' ', '%20')
        df['link'] = 'https://sistemasweb.b3.com.br/PlantaoNoticias/Noticias/Detail?idNoticia=' + df['NwsMsg.id'].astype(str) + '&agencia=' + df['NwsMsg.IdAgencia'].astype(str) + '&dataNoticia=' + df['NwsMsg.dateTime'].astype(str)
        
        for index, row in df.iterrows():
            url = row['link']
            response = requests.get(url)
            html_content = response.text
            soup = BeautifulSoup(html_content, 'html.parser')
            pre_element = soup.find('pre', id='conteudoDetalhe')
            pre_text = pre_element.get_text() if pre_element else ''
            padrao = r'(https?://[^&]+)'
            resultado = re.search(padrao, pre_text)
            if resultado:
                url_atualizada = resultado.group(1).replace('visualizarDocumento', 'exibirDocumento').split('&')[0]
                df.at[index, 'link'] = url_atualizada

        df.columns = ['IdAgencia', 'content', 'dateTime', 'headline', 'id', 'dia_hora', 'link']
        
        return df

In [18]:
import re
from datetime import datetime

fii_codigo = 'MXRF11'

ano = 2024
mes = 1
texto = 'Informe Mensal'
self = FII(fii_codigo, ano, mes)

self.url_noticia(texto)[1][0]

'2024-01-16'

In [5]:
fii_codigo = 'MXRF11'

ano = 2024
mes = 1

self = FII(fii_codigo, ano, mes)

texto = 'Informe Mensal'
elementos = 'table'
tabelas_fii = self.tabelas(texto, elementos)

df_dados_fundos = self.tabela(0, tabelas_fii)
df_cotistas = self.tabela(1, tabelas_fii)
df_ativos = self.tabela(2, tabelas_fii)
df_ativos2 = self.tabela(3, tabelas_fii)
df_passivos = self.tabela(4, tabelas_fii)

pattern = r'Mandato:\s*</b>(.*?)</span><span class="dado-cabecalho"><b>Segmento de Atuação:\s*</b>(.*?)</span><span class="dado-cabecalho"><b>Tipo de Gestão:\s*</b>(.*?)</span></td>'
match = re.search(pattern, str(tabelas_fii[0]))

df_fii = pd.DataFrame({
    'codigo_isin': [df_dados_fundos[self.indice(df_dados_fundos, 'Código ISIN')+1]],
    'cotistas': [self.str_para_int(df_cotistas[self.indice(df_cotistas, 'Número de cotista')+1])],
    'patrimonio_liquido': [self.str_para_float(df_ativos[self.indice(df_ativos, 'Patrimônio Líquido')+1])],
    'cotas_emitidas': [self.str_para_int(df_ativos[self.indice(df_ativos, 'Número de Cotas Emitidas')+1])],
    'valor_patrimonial_cota': [self.str_para_float(df_ativos[self.indice(df_ativos, 'Valor Patrimonial das Cotas')+1])],
    'valor_em_caixa': [self.str_para_float(df_ativos2[self.indice(df_ativos2, 'Total mantido para as Necessidades de Liquidez')+1])],
    'patrimonio_liquido': [self.str_para_float(df_ativos[self.indice(df_ativos, 'Patrimônio Líquido')+1])],
    'total_investido': [self.str_para_float(df_ativos2[self.indice(df_ativos2, 'Total investido')+1])],
    'passivo': [self.str_para_float(df_passivos[self.indice(df_passivos, 'Total do passivo')+1])],
    'mandato': [match.group(1).strip()],
    'seguimento': [match.group(2).strip()],
    'tipo_gestão': [match.group(3).strip()],
    'competencia': [df_dados_fundos[self.indice(df_dados_fundos, 'Competência')+1]],
})        
df_fii





,codigo_isin,cotistas,patrimonio_liquido,cotas_emitidas,valor_patrimonial_cota,valor_em_caixa,total_investido,passivo,mandato,seguimento,tipo_gestão,competencia
0,BRMXRFCTF008,1022791,2.851649e+09,287291278,9.925986,1.430485e+08,2.716371e+09,31106441.97,Híbrido,Híbrido,Ativa,12/2023


In [140]:
fii_codigo = 'XPML11'

ano = 2024
mes = 1

fii = FII(fii_codigo, ano, mes)
try:
    relatorios = fii.fii_relatorios()
    relatorios.to_csv(f'./csv/{fii_codigo}_relatorios_{ano}_{mes}.csv', sep=';', index=False)
except:
    pass

try:
    rendimentos = fii.fii_rendimento()
    rendimentos.to_csv(f'./csv/{fii_codigo}_rendimentos_{ano}_{mes}.csv', sep=';', index=False)
except:
    pass

try:
    informes = fii.fii_informe()
    informes.to_csv(f'./csv/{fii_codigo}_informes_{ano}_{mes}.csv', sep=';', index=False)
except:
    pass

In [134]:
fiis = ['HGLG11', 'XPML11', 'SNAG11']

anos = [2023, 2022]
meses = [12, 11, 10, 9, 8, 7, 6, 5, 4, 3, 2, 1]

for fii_codigo in fiis:
    for ano in anos:
        for mes in meses:
            fii = FII(fii_codigo, ano, mes)

            try:
                relatorios = fii.fii_relatorios()
                relatorios.to_csv(f'./csv/{fii_codigo}_relatorios_{ano}_{mes}.csv', sep=';', index=False)
            except:
                pass

            try:
                rendimentos = fii.fii_rendimento()
                rendimentos.to_csv(f'./csv/{fii_codigo}_rendimentos_{ano}_{mes}.csv', sep=';', index=False)
            except:
                pass

            try:
                informes = fii.fii_informe()
                informes.to_csv(f'./csv/{fii_codigo}_informes_{ano}_{mes}.csv', sep=';', index=False)
            except:
                pass

In [3]:
#https://fnet.bmfbovespa.com.br/fnet/publico/visualizarDocumento?id=587379
#https://fnet.bmfbovespa.com.br/fnet/publico/visualizarDocumento?id=588007

In [4]:
import yfinance as yf

fii_codigo = 'XPML11'

fii = yf.Ticker(fii_codigo+'.SA')

preco = fii.history(period="1d")["Close"].iloc[-1]

preco


117.19999694824219

In [22]:
from yahoo_fin import stock_info

fii_codigo = 'XPML11'
def fii_preco(fii_codigo):
    ticker = fii_codigo+'.SA'
    preco = stock_info.get_live_price(ticker)
    return "{:.2f}".format(preco)

fii_preco(fii_codigo)




'116.61'

In [29]:
def fii_preco(fii_codigo):
    ticker = fii_codigo+'.SA'
    preco = stock_info.get_live_price(ticker)
    atual = "{:.2f}".format(preco)

    hist = yf.download(ticker, period="1y")["Adj Close"]
    menor = "{:.2f}".format(hist.min())
    maior = "{:.2f}".format(hist.max())

    return atual, menor, maior


a, b, c = fii_preco('XPML11')

print(a,b,c)





[*********************100%%**********************]  1 of 1 completed

117.67 87.59 119.70


In [160]:
def df_csv_informes(fii_codigo):
    import glob
    import pandas as pd

    diretorio = './csv/'
    padrao = diretorio + f'/{fii_codigo}_rendimentos_*.csv'
    padrao = diretorio + f'/{fii_codigo}_informes_*.csv'
    dataframes = []

    for arquivo in glob.glob(padrao):
        df = pd.read_csv(arquivo, sep=';')
        dataframes.append(df)

    resultado = pd.concat(dataframes, ignore_index=True)

    return resultado

In [161]:
df_csv_informes(fii_codigo)

,codigo_isin,cotistas,patrimonio_liquido,cotas_emitidas,valor_patrimonial_cota,valor_em_caixa,total_investido,passivo,mandato,seguimento,tipo_gestão,competencia
0,BRMXRFCTF008,977470,2.574424e+09,259279007,9.929164,6.511452e+07,2.610877e+09,1.251801e+08,Híbrido,Híbrido,Ativa,09/2023
1,BRMXRFCTF008,988568,2.548277e+09,259279007,9.828320,1.119751e+08,2.503954e+09,9.300089e+07,Híbrido,Híbrido,Ativa,10/2023
2,BRMXRFCTF008,999290,2.555879e+09,259279007,9.857640,3.771702e+07,2.587478e+09,9.366335e+07,Híbrido,Híbrido,Ativa,11/2023
3,BRMXRFCTF008,777034,2.286074e+09,226039663,10.113598,6.738742e+07,2.292959e+09,1.086573e+08,Híbrido,Híbrido,Ativa,01/2023
4,BRMXRFCTF008,792703,2.276988e+09,226039663,10.073399,7.625362e+07,2.279627e+09,1.122729e+08,Híbrido,Híbrido,Ativa,02/2023
5,BRMXRFCTF008,808458,2.276171e+09,226039663,10.069787,8.513679e+07,2.270320e+09,1.115006e+08,Híbrido,Híbrido,Ativa,03/2023
6,BRMXRFCTF008,823365,2.284781e+09,226039663,10.107878,2.473227e+07,2.361422e+09,1.326130e+08,Híbrido,Híbrido,Ativa,04/2023
7,BRMXRFCTF008,843893,2.274794e+09,226039663,10.063694,8.685885e+07,2.292945e+09,1.352848e+08,Híbrido,Híbrido,Ativa,05/2023
8,BRMXRFCTF008,871906,2.283035e+09,226039663,10.100153,3.611937e+07,2.384966e+09,1.672918e+08,Híbrido,Híbrido,Ativa,06/2023
9,BRMXRFCTF008,908249,2.592331e+09,259301607,9.997357,1.368637e+08,2.577996e+09,1.728919e+08,Híbrido,Híbrido,Ativa,07/2023


,codigo_isin,cotistas,patrimonio_liquido,cotas_emitidas,valor_patrimonial_cota,valor_em_caixa,total_investido,passivo,mandato,seguimento,tipo_gestão,competencia
0,BRXPMLCTF000,370163,3.463638e+09,30822213,112.374738,8.173235e+08,3.299733e+09,7.270256e+08,Renda,Shoppings,Ativa,09/2023
1,BRXPMLCTF000,384432,3.455928e+09,30822213,112.124592,6.828539e+08,3.435761e+09,7.437977e+08,Renda,Shoppings,Ativa,10/2023
2,BRXPMLCTF000,396609,3.449365e+09,30822213,111.911660,5.803838e+08,3.569726e+09,7.922875e+08,Renda,Shoppings,Ativa,11/2023
3,BRXPMLCTF000,300598,2.151430e+09,21314099,100.939308,1.744040e+08,2.707205e+09,7.695707e+08,Renda,Shoppings,Ativa,01/2023
4,BRXPMLCTF000,304996,2.113026e+09,21314099,99.137461,1.176083e+08,2.669795e+09,7.562684e+08,Renda,Shoppings,Ativa,02/2023
5,BRXPMLCTF000,308245,2.101488e+09,21314099,98.596144,7.161376e+07,2.744380e+09,7.953466e+08,Renda,Shoppings,Ativa,03/2023
6,BRXPMLCTF000,312556,2.105159e+09,21314099,98.768368,5.732038e+07,2.758725e+09,7.959734e+08,Renda,Shoppings,Ativa,04/2023
7,BRXPMLCTF000,317438,2.098003e+09,21314099,98.432633,4.637410e+07,2.768434e+09,8.035539e+08,Renda,Shoppings,Ativa,05/2023
8,BRXPMLCTF000,325070,2.472954e+09,25117345,98.456012,3.528509e+08,2.836517e+09,8.049597e+08,Renda,Shoppings,Ativa,06/2023
9,BRXPMLCTF000,336875,3.050728e+09,26537865,114.957552,4.001863e+08,3.281935e+09,7.206226e+08,Renda,Shoppings,Ativa,07/2023


In [143]:
import glob
import pandas as pd

diretorio = './csv/'
padrao = diretorio + '/XPML11_rendimentos_*.csv'
dataframes = []

for arquivo in glob.glob(padrao):
    df = pd.read_csv(arquivo, sep=';')
    dataframes.append(df)

resultado = pd.concat(dataframes, ignore_index=True)

resultado

,codigo_isin,codigo_negociacao,proventos,data_base,data_pagamento,periodo
0,BRXPMLCTF000,XPML11,0.85,18/10/2023,25/10/2023,09/2023
1,BRXPMLCTF000,XPML11,0.85,17/11/2023,24/11/2023,10/2023
2,BRXPMLCTF000,XPML11,0.90,15/12/2023,22/12/2023,11/2023
3,BRXPMLCTF000,XPML11,0.77,15/02/2023,24/02/2023,01/2023
4,BRXPMLCTF000,XPML11,0.78,17/03/2023,24/03/2023,02/2023
5,BRXPMLCTF000,XPML11,0.81,18/05/2023,25/05/2023,04/2023
6,BRXPMLCTF000,XPML11,0.81,16/06/2023,23/06/2023,05/2023
7,BRXPMLCTF000,XPML11,0.83,18/07/2023,25/07/2023,06/2023
8,BRXPMLCTF000,XPML11,0.84,18/08/2023,25/08/2023,07/2023
9,BRXPMLCTF000,XPML11,0.85,18/09/2023,25/09/2023,08/2023


In [ ]:
rendimentos

,codigo_isin,codigo_negociacao,proventos,data_base,data_pagamento,periodo
0,BRSNAGR07M11,SNAG17,0.105,23/01/2024,25/01/2024,DEZEMBRO


In [ ]:
informes

,codigo_isin,cotistas,patrimonio_liquido,cotas_emitidas,valor_patrimonial_cota,valor_em_caixa,total_investido,passivo,mandato,seguimento,tipo_gestão,competencia
0,BRSNAGCTF000,78093,5.028591e+08,50000000,10.057183,28247273.12,4.806811e+08,6452467.62,Híbrido,Híbrido,Ativa,12/2023


In [ ]:
relatorios['link'][0]

'https://fnet.bmfbovespa.com.br/fnet/publico/exibirDocumento?id=588838'

In [ ]:
rendimentos

,codigo_isin,codigo_negociacao,proventos,data_base,data_pagamento,periodo
0,BRSNAGR07M11,SNAG17,0.105,23/01/2024,25/01/2024,DEZEMBRO


In [ ]:
fii_codigo = 'SNAG11'
fii = FII(fii_codigo, ano, mes)
fii.fii_relatorios()

,NwsMsg.IdAgencia,NwsMsg.content,NwsMsg.dateTime,NwsMsg.headline,NwsMsg.id,link
0,18,None,2024-01-23%2017:38:55,FIAGRO SUNO (SNAG) Aviso aos Cotistas - 23/01...,2454737,https://fnet.bmfbovespa.com.br/fnet/publico/ex...
1,18,None,2024-01-16%2016:11:12,FIAGRO SUNO (SNAG) Informe Mensal - 12/2023,2448571,https://fnet.bmfbovespa.com.br/fnet/publico/ex...
2,18,None,2024-01-15%2013:06:10,FIAGRO SUNO (SNAG) Aviso aos Cotistas - 15/01...,2446902,https://fnet.bmfbovespa.com.br/fnet/publico/ex...
3,18,None,2024-01-15%2011:26:38,FIAGRO SUNO (SNAG) Aviso aos Cotistas - 15/01...,2446753,https://fnet.bmfbovespa.com.br/fnet/publico/ex...
4,18,None,2024-01-15%2011:11:42,FIAGRO SUNO (SNAG) Aviso aos Cotistas - 15/01...,2446733,https://fnet.bmfbovespa.com.br/fnet/publico/ex...
5,18,None,2024-01-15%2009:55:01,FIAGRO SUNO (SNAG) Aviso aos Cotistas - 15/01...,2446596,https://fnet.bmfbovespa.com.br/fnet/publico/ex...


# Testes

In [ ]:
texto = 'Informe Mensal'
elementos = 'table'
tabelas_fii = self.tabelas(texto, elementos)

#def tabela(i):
#    tabela = tabelas_fii[i].find_all('td')
#    info = [item.text for item in tabela]
#    info = [info.strip() for info in info]
#    return info

df_dados_fundos = self.tabela(0, tabelas_fii)
df_cotistas = self.tabela(1, tabelas_fii)
df_ativos = self.tabela(2, tabelas_fii)
df_ativos2 = self.tabela(3, tabelas_fii)
df_passivos = self.tabela(4, tabelas_fii)

In [ ]:

relatorios = self.json()
df = pd.json_normalize(relatorios)
df['NwsMsg.dateTime'] = df['NwsMsg.dateTime'].str.replace(' ', '%20')
df['link'] = 'https://sistemasweb.b3.com.br/PlantaoNoticias/Noticias/Detail?idNoticia=' + df['NwsMsg.id'].astype(str) + '&agencia=' + df['NwsMsg.IdAgencia'].astype(str) + '&dataNoticia=' + df['NwsMsg.dateTime'].astype(str)

for index, row in df.iterrows():
    url = row['link']
    response = requests.get(url)
    html_content = response.text
    soup = BeautifulSoup(html_content, 'html.parser')
    pre_element = soup.find('pre', id='conteudoDetalhe')
    pre_text = pre_element.get_text() if pre_element else ''
    padrao = r'(https?://[^&]+)'
    resultado = re.search(padrao, pre_text)
    if resultado:
        url_atualizada = resultado.group(1).replace('visualizarDocumento', 'exibirDocumento').split('&')[0]
        df.at[index, 'link'] = url_atualizada
    


0.92

In [ ]:
# evolução:
## patrimonio liquido
## cotação 


# FIM

In [ ]:
class FIIExtractor:
    def __init__(self, fii_codigo):
        self.fii_codigo = fii_codigo

    def xhr_link(self, i):
        df_fundos = pd.read_csv('fundos.csv')    
        fundo = df_fundos[df_fundos['FII'] == self.fii_codigo[:4]]['URL']
        link_relatorios_fundo = fundo.iloc[i]
        return link_relatorios_fundo
    
    def json_api(self, link_requisicao):
        response = requests.get(link_requisicao)
        return response.json()
    
    def soup_webdriver(self, link):
        driver = webdriver.Chrome()
        driver.get(link)
        driver.implicitly_wait(5)
        html = driver.page_source
        driver.quit()
        return BeautifulSoup(html, 'html.parser')
    
    def documento_link(self, texto):
        item = [item for item in self.json_api(self.xhr_link(1)) if texto in item['headline']]
        url = item[0]['url']
        response = requests.get(url)
        html = response.text
        soup = BeautifulSoup(html, 'html.parser')
        informe = soup.find('iframe')
        url_informe_mensal_doc = url.split('visualizarDocumento')[0] + informe['src'].split('&')[0]
        return url_informe_mensal_doc
    
    def tabelas(self, tabelas, i):
        tabela = tabelas[i].find_all('td')
        info = [item.text for item in tabela]
        info = [info.strip() for info in info]
        return info
    
    def indice(self, tabela, expressao):
        indice = None
        for i, linha in enumerate(tabela):
            if expressao in linha:
                indice = i
                break
        return indice
    
    def str_para_float(self, valor):
        return float(valor.replace(".", "").replace(",", "."))
    
    def str_para_int(self, valor):
        return int(valor.replace(".", "").replace(",", "."))
    
    def info_fii(self):
        df_tabelas = self.soup_webdriver(self.documento_link('Informe Mensal')).find_all('table')
        cotistas = self.tabelas(df_tabelas, 1)
        ativo = self.tabelas(df_tabelas, 2)
        ativo2 = self.tabelas(df_tabelas, 3)

        df_info_fii = pd.DataFrame({
            'cotistas': [self.str_para_int(cotistas[self.indice(cotistas, 'detalhamento do número de cotistas')+3])],
            'ativo': ["{:.2f}".format(self.str_para_float(ativo[self.indice(ativo, 'Ativo – R$')+1]))],
            'patrimonio': ["{:.2f}".format(self.str_para_float(ativo[self.indice(ativo, 'Patrimônio Líquido – R$')+1]))],
            'cotas_emitidas': [self.str_para_float(ativo[self.indice(ativo, 'Cotas Emitidas')+1])],
            'valor_patrimonial_cota': [self.str_para_float(ativo[self.indice(ativo, 'Valor Patrimonial das Cotas')+1])],
            'valor_em_caixa': ["{:.2f}".format(self.str_para_float(ativo2[self.indice(ativo2, 'Total mantido para as Necessidades de Liquidez')+1]))],
        })
        
        return df_info_fii
    
    def dados_basicos_fii(self):
        link_requisicao = self.xhr_link(0) #dados basicos
        dados_fii = self.json_api(link_requisicao)
        
        col_acronym = dados_fii['detailFund']['acronym']
        col_tradingName = dados_fii['detailFund']['tradingName']
        col_tradingCode = dados_fii['detailFund']['tradingCode'].strip()
        col_cnpj = dados_fii['detailFund']['cnpj']
        col_classification = dados_fii['detailFund']['classification']
        col_website = dados_fii['detailFund']['webSite']

        df_dados_fundos = pd.DataFrame({
            'acronym': [col_acronym],
            'tradingName': [col_tradingName],
            'tradingCode': [col_tradingCode],
            'cnpj': [col_cnpj],
            'classification': [col_classification],
            'webSite': [col_website],
        })

        return df_dados_fundos

    def extract(self):
        return pd.concat([self.dados_basicos_fii(), self.info_fii()], axis=1)

# Example of usage:
fii_codigo = "XPML11"


In [ ]:
fii_extractor = FIIExtractor(fii_codigo)